# 1. If you were to continue working on this task, is there any idea you can use to improve the results of any model from the second section?

If I were to continue working on this I would:

## Data Understanding

- **Enhance Visualizations:** Improving the visualizations through cohesive styling will make the data easier to interpret. Incorporating interactivity into these visualizations can also allow users to explore the data more effectively.

- **Explore Variable Correlations:** Investigating correlations between various variables can reveal how they relate to each other, potentially uncovering hidden insights that could inform model improvements.
To enhance the results of the models in the second section, several strategies can be implemented:

## Model Performance
- **Integrate Weather Data:** Incorporating weather conditions as additional features can provide insights into how different weather patterns affect taxi demand and trip durations. This could lead to a more nuanced understanding of travel behaviors.

- **Incorporate Tourist Destinations:** Adding the locations of common tourist attractions and their respective taxi pickup points can help in understanding how tourism influences traffic patterns and taxi demand in different areas.

- **Test Additional Models:** Exploring and testing more models, such as conducting a quick AutoGluon search (results below), may uncover models that perform better than the initial baseline constructed. This can lead to more robust predictions.

- **Increase Computational Resources:** Currently limited by the capabilities of a personal laptop, increasing computational power would enable more extensive data analysis and modeling, facilitating the exploration of more complex models.

# 2. Do you have any finding or important conclusions that you were able to detect from the data?

- **Distance is Crucial for Fare:** The analysis shows that the distance of taxi trips has the greatest impact on fare costs. This suggests that optimizing routes for longer distances could enhance profitability for taxi services.

- **Traffic Patterns Influenced by Commuting:** While tourism does contribute to traffic congestion, the data indicates that work-related commutes are a more significant factor. This highlights the need for urban planning that accommodates both tourism and daily commuting patterns to alleviate congestion effectively.

The overall insights emphasize the dynamic nature of urban mobility and the importance of understanding various factors that influence taxi demand, fare structures, and traffic patterns. These findings can guide effective transportation planning and improve service optimization for taxi companies.


# Libs

In [24]:
!pip install --upgrade mxnet
!pip install autogluon
!pip install --upgrade autogluon.tabular
!pip install dask[dataframe]

In [2]:
import autogluon.tabular as task # TabularPrediction is now deprecated and part of autogluon.tabular

In [3]:
import autogluon as ag

In [6]:
import numpy as np

# Tab Data
import pandas as pd

# Geo Data
import geopandas as gpd
from shapely.geometry import shape

# Plotting
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import plotly.express as px
import plotly.io as pio

# xgboost
import xgboost as xgb

# hyperopt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from hyperopt.early_stop import no_progress_loss

# sklearn
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [7]:
import local_functions.features as f
import local_functions.preprocessing as p

# Functinos

In [8]:
def estimate_distance(x):
    global distances
    global geo_df

    key = "->".join([str(i) for i in x])

    point1 = x[0]
    point2 = x[1]

    if key in distances.keys():
        return distances[key]

    known_points = geo_df.LocationID.values

    if point1 not in known_points or point2 not in known_points:
        distances[key] = 0
        return 0
    else:
        df = (
            geo_df.query(f"LocationID == {point1}")
            .distance(geo_df.query(f"LocationID == {point2}"), align=False)
            .reset_index()
            .rename(columns={0: "estimated_distance"})
        )
        distances[key] = df.estimated_distance.values[0]
        return distances[key]

# Data

## Geo Data

In [9]:
geo_df = gpd.read_file("data/taxi_zones_shapefile/taxi_zones.shp")
geo_df["area_mi2"] = geo_df.geometry.apply(lambda x: shape(x).area / 2.788e7)
area_map = geo_df.set_index("LocationID")["area_mi2"].to_dict()
area_map[264] = 0  # Fix missing value
geo_df.head()

OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  area_mi2  
0            EWR  POLYGON ((933100.918 192536.086, 933091.011 19...  2.834990  
1         Queens  MULTIPOLYGON (((1033269.244 172126.008, 103343...  5.161748  
2          Bronx  POLYGON ((1026308.77 256767.698, 1026495.593 2...  1.136481  
3      Manhattan  POLYGON ((992073.467 203714.076, 992068.667 20...  0.287795  
4  Staten Island  POLYGON ((935843.31 144283.336, 936046.565 144...  1.808281

In [10]:
geo_df["centroid"] = geo_df.centroid

geo_df = geo_df.sort_values("area_mi2", ascending=False).drop_duplicates(
    "LocationID", keep="first"
)

## Trip Data

In [11]:
trip_features = [
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "tip_amount",
]

drop_features = ["store_and_fwd_flag"]

LOW_QUANTILE = 0.01
HIGH_QUANTILE = 0.95

us_holidays = p.get_us_holidays(2023)

In [12]:
df = pd.read_csv(
    "data/yellow_taxi_trip_records_2023.csv",
)

<ipython-input-12-7bad0566ca6b>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [13]:
df = (
    df.drop(columns=drop_features)
    .pipe(p.normalize_columns)
    .pipe(p.col_to_datetime)
    .pipe(p.valid_vendor_payment_filter)
    .pipe(p.valid_trip_filter, cols=trip_features)
    .pipe(p.quantile_filter, cols=trip_features, low=LOW_QUANTILE, high=HIGH_QUANTILE)
    .pipe(p.impute_missing)
    .pipe(f.calculate_time_features, us_holidays)
    .pipe(f.calculate_trip_features)
)

## Estimating distance

In [14]:
keep_features = [
    "vendor_id",
    "passenger_count",
    "pu_location_id",
    "do_location_id",
    "tpep_pickup_month",
    "tpep_pickup_hour",
    "tpep_pickup_day",
    "tpep_pickup_weekday",
    "holiday",
    "fare_amount",
]
df = df[keep_features]

In [15]:
df["from_to"] = df[["pu_location_id", "do_location_id"]].apply(list, axis=1)
df["from_to"] = df["from_to"].apply(lambda x: sorted(x))

distances = {}

In [16]:
df["estimated_distance"] = df["from_to"].apply(lambda x: estimate_distance(x))

In [17]:
df.drop(columns=["from_to"], inplace=True)

In [18]:
df = df.sample(frac = .1)

# Training

In [19]:
def split_data(df, train_fraction):
    """Splits a DataFrame into training and testing sets.

    Args:
      df: The DataFrame to split.
      train_fraction: The fraction of data to use for training.

    Returns:
      A tuple containing the training and testing DataFrames.
    """
    train_df = df.sample(frac=train_fraction, random_state=42) # Use random_state for reproducibility
    test_df = df.drop(index=train_df.index)
    return train_df, test_df

In [20]:
train_df, test_df = split_data(df, train_fraction=0.8)

In [21]:
from autogluon.tabular import TabularPredictor

# Assuming 'task' was meant to be a TabularPredictor
predictor = TabularPredictor(label='fare_amount') # Create a TabularPredictor object
predictor = predictor.fit(train_data=train_df,presets='medium_quality',time_limit=60*30) # Call the fit method on the predictor object
performance = predictor.evaluate(test_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20240928_165720"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.86 GB / 12.67 GB (77.8%)
Disk Space Avail:   4.81 GB / 15.00 GB (32.1%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "AutogluonModels/ag-20240928_165720"
Train Data Rows:    426272
Train Data Columns: 10
Label Column:       fare_amount
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (41.5, 5.1, 13.9599, 6.6033)

[1000]	valid_set's rmse: 3.54153
[2000]	valid_set's rmse: 3.45968
[3000]	valid_set's rmse: 3.41689
[4000]	valid_set's rmse: 3.38799
[5000]	valid_set's rmse: 3.36527
[6000]	valid_set's rmse: 3.34808
[7000]	valid_set's rmse: 3.33495
[8000]	valid_set's rmse: 3.32531
[9000]	valid_set's rmse: 3.32005
[10000]	valid_set's rmse: 3.31085


	-3.3108	 = Validation score   (-root_mean_squared_error)
	296.94s	 = Training   runtime
	4.9s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 1487.05s of the 1487.04s of remaining time.


[1000]	valid_set's rmse: 3.28832
[2000]	valid_set's rmse: 3.23085
[3000]	valid_set's rmse: 3.20566
[4000]	valid_set's rmse: 3.19545
[5000]	valid_set's rmse: 3.1942
[6000]	valid_set's rmse: 3.19458


	-3.1923	 = Validation score   (-root_mean_squared_error)
	143.18s	 = Training   runtime
	1.84s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 1340.93s of the 1340.92s of remaining time.
	-3.325	 = Validation score   (-root_mean_squared_error)
	560.54s	 = Training   runtime
	0.87s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 770.88s of the 770.87s of remaining time.
	Ran out of time, early stopping on iteration 8784.
	-3.2115	 = Validation score   (-root_mean_squared_error)
	770.92s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the -0.36s of remaining time.
	Ensemble Weights: {'LightGBM': 0.7, 'CatBoost': 0.2, 'KNeighborsDist': 0.1}
	-3.185	 = Validation score   (-root_mean_squared_error)
	0.02s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1800.7s ... Best model: WeightedEnsemble_L2 | Estimated 

In [22]:
predictor.leaderboard(test_df)

model  score_test  score_val              eval_metric  \
0  WeightedEnsemble_L2   -3.136305  -3.185037  root_mean_squared_error   
1             LightGBM   -3.140068  -3.192345  root_mean_squared_error   
2             CatBoost   -3.152266  -3.211526  root_mean_squared_error   
3           LightGBMXT   -3.260901  -3.310781  root_mean_squared_error   
4      RandomForestMSE   -3.282530  -3.325035  root_mean_squared_error   
5       KNeighborsUnif   -3.569323  -3.560013  root_mean_squared_error   
6       KNeighborsDist   -3.614560  -3.600940  root_mean_squared_error   

   pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  \
0       45.739732       2.069906  916.038712                 0.024330   
1       40.043884       1.839720  143.178777                40.043884   
2        1.574654       0.039866  770.923710                 1.574654   
3      118.116958       4.896512  296.940497               118.116958   
4        9.518188       0.865903  560.539552                 9.518188   
5        4.010133       0.183614    6.363916                 4.010133   
6        4.096863       0.189249    1.917759                 4.096863   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.001071           0.018465            2       True   
1                1.839720         143.178777            1       True   
2                0.039866         770.923710            1       True   
3                4.896512         296.940497            1       True   
4                0.865903         560.539552            1       True   
5                0.183614           6.363916            1       True   
6                0.189249           1.917759            1       True   

   fit_order  
0          7  
1          4  
2          6  
3          3  
4          5  
5          1  
6          2

In [23]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val              eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -3.185037  root_mean_squared_error       2.069906  916.038712                0.001071           0.018465            2       True          7
1             LightGBM  -3.192345  root_mean_squared_error       1.839720  143.178777                1.839720         143.178777            1       True          4
2             CatBoost  -3.211526  root_mean_squared_error       0.039866  770.923710                0.039866         770.923710            1       True          6
3           LightGBMXT  -3.310781  root_mean_squared_error       4.896512  296.940497                4.896512         296.940497            1       True          3
4      RandomForestMSE  -3.325035  root_mean_squared_error       0.865903  560.539552                0.865903         

{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'LightGBMXT': 'LGBModel',
  'LightGBM': 'LGBModel',
  'RandomForestMSE': 'RFModel',
  'CatBoost': 'CatBoostModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': -3.5600134981362532,
  'KNeighborsDist': -3.60093994979056,
  'LightGBMXT': -3.310780916103169,
  'LightGBM': -3.1923449735795306,
  'RandomForestMSE': -3.3250346290414297,
  'CatBoost': -3.211526086038219,
  'WeightedEnsemble_L2': -3.185036756620344},
 'model_best': 'WeightedEnsemble_L2',
 'model_paths': {'KNeighborsUnif': ['KNeighborsUnif'],
  'KNeighborsDist': ['KNeighborsDist'],
  'LightGBMXT': ['LightGBMXT'],
  'LightGBM': ['LightGBM'],
  'RandomForestMSE': ['RandomForestMSE'],
  'CatBoost': ['CatBoost'],
  'WeightedEnsemble_L2': ['WeightedEnsemble_L2']},
 'model_fit_times': {'KNeighborsUnif': 6.363915681838989,
  'KNeighborsDist': 1.9177591800689697,
  'LightGBMXT': 296.9404969215393,
  'LightGBM'